In [1]:
import torch
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.optim.lr_scheduler as lr_scheduler
import pandas as pd
import numpy as np

In [2]:
class SignLanguageCNN(nn.Module):
    def __init__(self, num_classes=25):
        super(SignLanguageCNN, self).__init__()
        
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.Conv2d(32, 32, kernel_size=5, padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2),
            nn.Dropout(0.25)
        )

        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.Conv2d(64, 64, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2, stride=2),
            nn.Dropout(0.25)
        )

        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.Conv2d(128, 128, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2, stride=2),
            nn.Dropout(0.25)
        )

        self.layer4 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.Conv2d(256, 256, kernel_size=3, padding='same'),
            nn.ReLU(),
            nn.BatchNorm2d(256),
            nn.MaxPool2d(2, stride=2),
            nn.Dropout(0.25)
        )

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(256 * 1 * 1, 512)  # Correctly adjusted for the flattened size
        self.fc2 = nn.Linear(512, num_classes)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, 0.3)
        x = self.fc2(x)
        return x

In [17]:
# Initialize your model
model = SignLanguageCNN()

# Load the saved weights into the model
weights_path = '../data/weights/2_0.9881024644894986_py_torch_cnn.pkl'
checkpoint = torch.load(weights_path)
model.load_state_dict(checkpoint)

<All keys matched successfully>

In [18]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import TensorDataset, DataLoader

# Assuming you have loaded your data as you've shown
old_test_df = pd.read_csv("../data/raw/old_sign_mnist_test.csv")

test_labels = old_test_df['label'].values
test_images = old_test_df.drop('label', axis=1).values
test_images = test_images.reshape(-1, 1, 28, 28).astype(np.float32)  

labels_tensor = torch.tensor(test_labels).long()
images_tensor = torch.tensor(test_images)

# Creating TensorDataset for the test set
test_dataset = TensorDataset(images_tensor, labels_tensor)  # Adjust here

# Creating DataLoader for the test set
batch_size = 256
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

# Now iterate through the test_loader for making predictions
predictions = []
with torch.no_grad():
    for images, _ in test_loader: 
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)  # Get the predicted classes
        predictions.extend(predicted.tolist())  # Append predictions to a list


In [20]:
np.array(predictions)


array([ 2, 14, 14, ..., 20, 12,  1])

In [21]:
test_df['label'].values

array([ 6,  5, 10, ...,  2,  4,  2])